<a href="https://colab.research.google.com/github/thanit456/Optuna_tutorial/blob/master/optuna_integrate_tf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install optuna

# First optimization

In [5]:
import optuna

In [6]:
def objective(trial):
  x = trial.suggest_uniform('x', -10, 10)
  return (x - 2) ** 2

In [8]:
study = optuna.create_study()
study.optimize(objective, n_trials=100)

[I 2020-06-18 14:19:03,012] Finished trial#0 with value: 113.26112440667953 with parameters: {'x': -8.642420984281703}. Best is trial#0 with value: 113.26112440667953.
[I 2020-06-18 14:19:03,079] Finished trial#1 with value: 26.095832196694516 with parameters: {'x': -3.108407990430533}. Best is trial#1 with value: 26.095832196694516.
[I 2020-06-18 14:19:03,136] Finished trial#2 with value: 1.1904372336022886 with parameters: {'x': 0.9089284012484384}. Best is trial#2 with value: 1.1904372336022886.
[I 2020-06-18 14:19:03,192] Finished trial#3 with value: 41.593137485088796 with parameters: {'x': -4.449274182812264}. Best is trial#2 with value: 1.1904372336022886.
[I 2020-06-18 14:19:03,249] Finished trial#4 with value: 21.581361506981416 with parameters: {'x': 6.6455744001125865}. Best is trial#2 with value: 1.1904372336022886.
[I 2020-06-18 14:19:03,305] Finished trial#5 with value: 33.61282013269059 with parameters: {'x': 7.797656434516501}. Best is trial#2 with value: 1.190437233602

In [9]:
study.best_params

{'x': 1.9948423559580963}

In [10]:
study.best_trial

FrozenTrial(number=74, value=2.6601292062984364e-05, datetime_start=datetime.datetime(2020, 6, 18, 14, 19, 7, 391346), datetime_complete=datetime.datetime(2020, 6, 18, 14, 19, 7, 457908), params={'x': 1.9948423559580963}, distributions={'x': UniformDistribution(high=10, low=-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=74, state=TrialState.COMPLETE)

In [11]:
study.trials

[FrozenTrial(number=0, value=113.26112440667953, datetime_start=datetime.datetime(2020, 6, 18, 14, 19, 2, 937605), datetime_complete=datetime.datetime(2020, 6, 18, 14, 19, 3, 11725), params={'x': -8.642420984281703}, distributions={'x': UniformDistribution(high=10, low=-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=0, state=TrialState.COMPLETE),
 FrozenTrial(number=1, value=26.095832196694516, datetime_start=datetime.datetime(2020, 6, 18, 14, 19, 3, 15331), datetime_complete=datetime.datetime(2020, 6, 18, 14, 19, 3, 79017), params={'x': -3.108407990430533}, distributions={'x': UniformDistribution(high=10, low=-10)}, user_attrs={}, system_attrs={}, intermediate_values={}, trial_id=1, state=TrialState.COMPLETE),
 FrozenTrial(number=2, value=1.1904372336022886, datetime_start=datetime.datetime(2020, 6, 18, 14, 19, 3, 81666), datetime_complete=datetime.datetime(2020, 6, 18, 14, 19, 3, 136203), params={'x': 0.9089284012484384}, distributions={'x': UniformDistributio

# Adv. Configuration

In [12]:
def objective(trial):
  # Categorical parameter
  optimizer = trial.suggest_categorical('classifier', ['SVC', 'RandomForest'])

  # Int parameter
  num_layers = trial.suggest_int('num_layers', 1, 3)
  
  # Uniform parameter
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.0, 1.0)
  
  # Loguniform parameter
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
  
  # Discrete-uniform parameter
  drop_path_rate = trial.suggest_discrete_uniform('drop_path_rate', 0.0, 1.0, 0.1)


  # ....

# Optuna including TF 2.x estimator with MNIST


In [25]:
import shutil
import tempfile

import tensorflow as tf
import tensorflow_datasets as tfds

import optuna

MODEL_DIR = tempfile.mkdtemp()
BATCH_SIZE = 128
TRAIN_STEPS = 1000
PRUNING_INTERVAL_STEPS = 50
N_TRAIN_BATCHES = 3000
N_VALID_BATCHES = 1000


In [26]:
def preprocess(image, label):
    image = tf.reshape(image, [-1, 28 * 28])
    image = tf.cast(image, tf.float32)
    image /= 255
    label = tf.cast(label, tf.int32)
    return {"x": image}, label


def train_input_fn():
    data = tfds.load(name="mnist", as_supervised=True)
    train_ds = data["train"]
    train_ds = train_ds.map(preprocess).shuffle(60000).batch(BATCH_SIZE).take(N_TRAIN_BATCHES)
    return train_ds


def eval_input_fn():
    data = tfds.load(name="mnist", as_supervised=True)
    valid_ds = data["test"]
    valid_ds = valid_ds.map(preprocess).shuffle(10000).batch(BATCH_SIZE).take(N_VALID_BATCHES)
    return valid_ds

In [27]:
def create_classifier(trial):
    # We optimize the numbers of layers and their units.

    n_layers = trial.suggest_int("n_layers", 1, 3)
    hidden_units = []
    for i in range(n_layers):
        n_units = trial.suggest_int("n_units_l{}".format(i), 1, 128)
        hidden_units.append(n_units)

    config = tf.estimator.RunConfig(
        save_summary_steps=PRUNING_INTERVAL_STEPS, save_checkpoints_steps=PRUNING_INTERVAL_STEPS
    )

    model_dir = "{}/{}".format(MODEL_DIR, trial.number)
    classifier = tf.estimator.DNNClassifier(
        feature_columns=[tf.feature_column.numeric_column("x", shape=[28 * 28])],
        hidden_units=hidden_units,
        model_dir=model_dir,
        n_classes=10,
        optimizer=lambda: tf.keras.optimizers.Adam(learning_rate=0.01),
        config=config,
    )

    return classifier


In [29]:
def objective(trial):
    classifier = create_classifier(trial)

    optuna_pruning_hook = optuna.integration.TensorFlowPruningHook(
        trial=trial,
        estimator=classifier,
        metric="accuracy",
        run_every_steps=PRUNING_INTERVAL_STEPS,
    )

    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, max_steps=TRAIN_STEPS, hooks=[optuna_pruning_hook]
    )

    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, start_delay_secs=0, throttle_secs=0)

    eval_results, _ = tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

    return float(eval_results["accuracy"])


In [30]:
def study_optimize():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25)
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    shutil.rmtree(MODEL_DIR)

In [31]:
study_optimize()

INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9o7g5lzm/0', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 50, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmp9o7g5lzm/0', '_tf_random_seed': None, '_save_summary_steps': 50, '_save_checkpoints_steps': 50, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 469: /tmp/tmp9o7g5lzm/24/model.ckpt-469


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 469: /tmp/tmp9o7g5lzm/24/model.ckpt-469


INFO:tensorflow:Loss for final step: 0.25762323.


INFO:tensorflow:Loss for final step: 0.25762323.
[I 2020-06-18 16:48:45,036] Finished trial#24 with value: 0.9646999835968018 with parameters: {'n_layers': 1, 'n_units_l0': 114}. Best is trial#24 with value: 0.9646999835968018.


Study statistics: 
  Number of finished trials:  25
  Number of pruned trials:  9
  Number of complete trials:  16
Best trial:
  Value:  0.9646999835968018
  Params: 
    n_layers: 1
    n_units_l0: 114


# Optuna including TF 2.x estimator with images 

In [34]:
!pip install efficientnet

In [32]:
import shutil
import tempfile

import tensorflow as tf
import tensorflow_datasets as tfds

import optuna

MODEL_DIR = tempfile.mkdtemp()
BATCH_SIZE = 128
TRAIN_STEPS = 1000
PRUNING_INTERVAL_STEPS = 50
N_TRAIN_BATCHES = 3000
N_VALID_BATCHES = 1000

In [33]:
def preprocess(image, label):
    image = tf.reshape(image, [-1, 28 * 28])
    image = tf.cast(image, tf.float32)
    image /= 255
    label = tf.cast(label, tf.int32)
    return {"x": image}, label


def train_input_fn():
    data = tfds.load(name="mnist", as_supervised=True)
    train_ds = data["train"]
    train_ds = train_ds.map(preprocess).shuffle(60000).batch(BATCH_SIZE).take(N_TRAIN_BATCHES)
    return train_ds


def eval_input_fn():
    data = tfds.load(name="mnist", as_supervised=True)
    valid_ds = data["test"]
    valid_ds = valid_ds.map(preprocess).shuffle(10000).batch(BATCH_SIZE).take(N_VALID_BATCHES)
    return valid_ds

In [35]:
def efficientnetb4_teacher(input_shape = (250, 100, 3), num_classes = 5):
    # 285, 120 2.44 | 414, 163, 2.65
    # import sys
    # sys.path.append('model/')
    import efficientnet.tfkeras as efn
    backbone = efn.EfficientNetB4(input_shape = input_shape, weights='imagenet', include_top = False)
    pooler = tf.keras.layers.GlobalAveragePooling2D()(backbone.output)
    out = tf.keras.layers.Dense(num_classes)(pooler)
    softmax = tf.keras.layers.Activation('softmax')(out)

    model = tf.keras.models.Model(inputs = backbone.input, outputs = softmax)
    return model

In [ ]:
def create_classifier(trial):
    # We optimize the numbers of layers and their units.

    # n_layers = trial.suggest_int("n_layers", 1, 3)
    # hidden_units = []
    # for i in range(n_layers):
    #     n_units = trial.suggest_int("n_units_l{}".format(i), 1, 128)
    #     hidden_units.append(n_units)
    num_classes = 2

    keras_model = efficientnetb4_teacher(input_shape=(28, 28, 1), num_classes=num_classes)
    
    # ! it may have to model.commpile(...)
    
    config = tf.estimator.RunConfig(
        save_summary_steps=PRUNING_INTERVAL_STEPS, save_checkpoints_steps=PRUNING_INTERVAL_STEPS
    )

    model_dir = "{}/{}".format(MODEL_DIR, trial.number)

    classifier = tf.keras.estimator.model_to_estimator(
        keras_model=keras_model,
        custom_objects=None,
        model_dir=model_dir
        config=config
    )

    # classifier = tf.estimator.DNNClassifier(
    #     feature_columns=[tf.feature_column.numeric_column("x", shape=[28 * 28])],
    #     hidden_units=hidden_units,
    #     model_dir=model_dir,
    #     n_classes=num_classes,
    #     optimizer=lambda: tf.keras.optimizers.Adam(learning_rate=trial.suggest_loguniform("learning_rate", 1e-3, 1e-5)),
    #     config=config,
    # )

    return classifier


In [ ]:
def objective(trial):
    classifier = create_classifier(trial)

    optuna_pruning_hook = optuna.integration.TensorFlowPruningHook(
        trial=trial,
        estimator=classifier,
        metric="accuracy",
        run_every_steps=PRUNING_INTERVAL_STEPS,
    )

    train_spec = tf.estimator.TrainSpec(
        input_fn=train_input_fn, max_steps=TRAIN_STEPS, hooks=[optuna_pruning_hook]
    )

    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn, start_delay_secs=0, throttle_secs=0)

    eval_results, _ = tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)

    return float(eval_results["accuracy"])


In [ ]:
def study_optimize():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=25)
    pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
    complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

    shutil.rmtree(MODEL_DIR)

In [ ]:
study_optimize()